# 수강생분의 이름, 학번을 반영해주세요.

In [1]:
id = '20197132'
name = '주준하'
print(id, name)

20197132 주준하


코랩 메뉴 -> 수정 -> 노트 설정 -> 하드웨어 가속기 GPU 권장(행렬 연산이 많기 때문)

구글 드라이브 연동

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


폴더 경로 설정

In [3]:
workspace_path = '/gdrive/My Drive/Colab Notebooks/CV_과제5'

필요 패키지 로드

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
import random
import numpy as np
import matplotlib.pyplot as plt

결과 재현을 위한 설정

In [5]:
seed = 1
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

합성곱 신경망(CNN) 정의

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 3x32x32 -> 10x32x32
            nn.BatchNorm2d(10),  # 배치 정규화
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 10x32x32 -> 20x32x32
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # 20x32x32 -> 20x16x16 (특징 압축)
            nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 20x16x16 -> 40x16x16
            nn.BatchNorm2d(40),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=40, out_channels=80, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 40x8x8 -> 80x8x8
            nn.BatchNorm2d(80),
            nn.AdaptiveAvgPool2d(1)  # 80x8x8 -> 80x1x1 (채널 별 평균값 계산)
        )
        self.fc = nn.Linear(80, 10)  # 출력값의 차원은 판별할 클래스 수인 10으로 설정 (CIFAR-10 10종 판별 문제)

    def forward(self, x):
        x = x.float()
        x = x.view(-1, 3, 32, 32)  # view 함수로 tensor 형태 변경: [batch크기, 3, 32, 32]
        x = self.main(x)  # CNN 모델 feed-forward
        x = x.view(-1, 80)  # view 함수로 형태 변경: [batch크기, 80]
        x = self.fc(x)  # 마지막 레이어에는 활성화 함수 사용하지 않음
        return x

print("init model done")

init model done


모델 학습을 위한 하이퍼파라미터 셋팅

In [7]:
batch_size = 64  # 학습 배치 크기
test_batch_size = 1000  # 테스트 배치 크기 (학습 과정을 제외하므로 더 큰 배치 사용 가능)
max_epochs = 10  # 학습 데이터셋 총 훈련 횟수
lr = 0.01  # 학습률
momentum = 0.5  # SGD에 사용할 모멘텀 설정 (파라미터 업데이트 시 관성 효과 사용)
log_interval = 200  # interval 때마다 로그 남김

use_cuda = torch.cuda.is_available()  # GPU cuda 사용 여부 확인

device = torch.device("cuda" if use_cuda else "cpu")  # GPU cuda 사용하거나 없다면 CPU 사용

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  # num_workers: data loading할 프로세스 수, pin_memory: 고정된 메모리 영역 사용

print("set vars and device done")

set vars and device done


데이터 로더 정의 (학습용, 테스트용 따로 정의)

In [8]:
transform = transforms.Compose([
                 transforms.ToTensor(),  # numpy array -> tensor 변환
                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 입력값 정규화 (일반적으로는 학습 데이터셋의 평균, 표준편차 사용)

# CIFAR-10 link: https://www.cs.toronto.edu/~kriz/cifar.html
# 학습용 데이터 로더 (CIFAR-10 학습 데이터셋 사용)
train_loader = torch.utils.data.DataLoader(
  datasets.CIFAR10(os.path.join(workspace_path, 'data'), train=True, download=True, 
                   transform=transform), 
    batch_size = batch_size, shuffle=True, drop_last=True, **kwargs)  # drop_last: 마지막 미니배치 크기가 batch_size 이하면 drop 

# 테스트용 데이터 로더 (CIFAR-10 테스트 데이터셋 사용)
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(os.path.join(workspace_path, 'data'), train=False, download=True,
                         transform=transform), 
    batch_size=test_batch_size, shuffle=False, **kwargs)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /gdrive/My Drive/Colab Notebooks/data/cifar-10-python.tar.gz to /gdrive/My Drive/Colab Notebooks/data
Files already downloaded and verified


모델, 최적화 알고리즘, 손실 함수 정의

In [9]:
model = Net().to(device)  # 모델 정의
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # 최적화 알고리즘 정의 (SGD 사용)
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의 (CrossEntropy 사용)

AverageMeter 정의

In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

학습, 테스트용 함수 정의

In [11]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()  # 모델 학습 모드 설정
    summary_loss = AverageMeter()  # 학습 손실값 기록 초기화
    summary_acc = AverageMeter() # 학습 정확도 기록 초기화
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 현재 미니 배치의 데이터, 정답 불러옴
        optimizer.zero_grad()  # gradient 0으로 초기화
        output = model(data)  # 모델에 입력값 feed-forward
        loss = criterion(output, target)  # 예측값(클래스 별 score)과 정답간의 손실값 계산
        loss.backward()  # 손실값 역전파 (각 계층에서 gradient 계산, pytorch는 autograd로 gradient 자동 계산)
        optimizer.step()  # 모델의 파라미터 업데이트 (gradient 이용하여 파라미터 업데이트)
        summary_loss.update(loss.detach().item())  # 손실값 기록
        pred = output.argmax(dim=1, keepdim=True)  # 예측값 중에서 최고 score를 달성한 클래스 선발
        correct = pred.eq(target.view_as(pred)).sum().item()  # 정답과 예측 클래스가 일치한 개수
        summary_acc.update(correct / data.size(0))  # 정확도 기록
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}, Accuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), summary_loss.avg, summary_acc.avg))
            
    return summary_loss.avg, summary_acc.avg

def test(log_interval, model, device, test_loader):
    model.eval()  # 모델 검증 모드 설정 (inference mode)
    summary_loss = AverageMeter()  # 테스트 손실값 기록 초기화
    summary_acc = AverageMeter() # 테스트 정확도 기록 초기화
    with torch.no_grad():  # 검증 모드이므로 gradient 계산안함
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # 현재 미니 배치의 데이터, 정답 불러옴
            output = model(data)  # 모델에 입력값 feed-forward
            loss = criterion(output, target)  # 예측값(클래스 별 score)과 정답간의 손실값 계산
            summary_loss.update(loss.detach().item())  # 손실값 기록
            pred = output.argmax(dim=1, keepdim=True)  # 예측값 중에서 최고 score를 달성한 클래스 선발
            correct = pred.eq(target.view_as(pred)).sum().item()  # 정답과 예측 클래스가 일치한 개수
            summary_acc.update(correct / data.size(0))  # 정확도 기록

    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.6f}\n'.format
          (summary_loss.avg, summary_acc.avg))  # 정답을 맞춘 개수 / 테스트셋 샘플 수 -> Accuracy

    return summary_loss.avg, summary_acc.avg

학습, 테스트, 모델 저장 수행

In [12]:
best_acc = 0
best_epoch = 0
for epoch in range(1, max_epochs+1):
    train_loss, train_acc = train(log_interval, model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(log_interval, model, device, test_loader)

    # 테스트에서 best accuracy 달성하면 모델 저장
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        torch.save(model, os.path.join(workspace_path, f'cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt'))
        print(f'# save model: cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt\n')

print(f'\n\n# Best accuracy model({best_acc * 100:.2f}%): cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt\n')

Train Epoch: 1 [0/50000 (0%)]	Average loss: 2.298636, Accuracy: 0.109375
Train Epoch: 1 [12800/50000 (26%)]	Average loss: 1.852039, Accuracy: 0.322217
Train Epoch: 1 [25600/50000 (51%)]	Average loss: 1.741262, Accuracy: 0.367869
Train Epoch: 1 [38400/50000 (77%)]	Average loss: 1.671902, Accuracy: 0.397047

Test set: Average loss: 1.4414, Accuracy: 0.489600

# save model: cifar10_cnn_model_best_acc_1-epoch.pt

Train Epoch: 2 [0/50000 (0%)]	Average loss: 1.511683, Accuracy: 0.546875
Train Epoch: 2 [12800/50000 (26%)]	Average loss: 1.407018, Accuracy: 0.497823
Train Epoch: 2 [25600/50000 (51%)]	Average loss: 1.386691, Accuracy: 0.507053
Train Epoch: 2 [38400/50000 (77%)]	Average loss: 1.376745, Accuracy: 0.510087

Test set: Average loss: 1.4488, Accuracy: 0.454000

Train Epoch: 3 [0/50000 (0%)]	Average loss: 1.257560, Accuracy: 0.515625
Train Epoch: 3 [12800/50000 (26%)]	Average loss: 1.262589, Accuracy: 0.550995
Train Epoch: 3 [25600/50000 (51%)]	Average loss: 1.256609, Accuracy: 0.55272

# 실습과제

## Accuracy 74% 이상 성능 달성하기
- 성능 개선 필수사항: 1) 네트워크 구조 개선 2) 데이터 증대 3) 하이퍼파라미터 수정
- baseline 모델 Accuracy: 63.67%
- torchvision: 데이터셋, 데이터증대 함수 제공 (https://pytorch.org/vision/stable/transforms.html)


개선 모델 성능: (Accuracy 74% 이상 달성한 성능 작성하기)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 3x32x32 -> 10x32x32
            nn.BatchNorm2d(10),  # 배치 정규화
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 10x32x32 -> 20x32x32
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # 20x32x32 -> 20x16x16 (특징 압축)
            nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 20x16x16 -> 40x16x16
            nn.BatchNorm2d(40),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=40, out_channels=20, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 20x16x16 -> 40x16x16
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 20x16x16 -> 40x16x16
            nn.BatchNorm2d(40),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=40, out_channels=60, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 20x16x16 -> 40x16x16
            nn.BatchNorm2d(60),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=60, out_channels=80, kernel_size=3,
                      stride=1, padding=1, bias=False),  # 40x8x8 -> 80x8x8
            nn.BatchNorm2d(80),
            nn.AdaptiveAvgPool2d(1)  # 80x8x8 -> 80x1x1 (채널 별 평균값 계산)
        )
        self.fc = nn.Linear(80, 10)  # 출력값의 차원은 판별할 클래스 수인 10으로 설정 (CIFAR-10 10종 판별 문제)

    def forward(self, x):
        x = x.float()
        x = x.view(-1, 3, 32, 32)  # view 함수로 tensor 형태 변경: [batch크기, 3, 32, 32]
        x = self.main(x)  # CNN 모델 feed-forward
        x = x.view(-1, 80)  # view 함수로 형태 변경: [batch크기, 80]
        x = self.fc(x)  # 마지막 레이어에는 활성화 함수 사용하지 않음
        return x

print("init model done")

init model done


In [29]:
batch_size = 64  # 학습 배치 크기
test_batch_size = 1000  # 테스트 배치 크기 (학습 과정을 제외하므로 더 큰 배치 사용 가능)
max_epochs = 20  # 학습 데이터셋 총 훈련 횟수
lr = 0.01  # 학습률

In [30]:
model = Net().to(device)  # 모델 정의
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # 최적화 알고리즘 정의 (SGD 사용)
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의 (CrossEntropy 사용)

In [31]:
best_acc = 0
best_epoch = 0
for epoch in range(1, max_epochs+1):
    train_loss, train_acc = train(log_interval, model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(log_interval, model, device, test_loader)

    # 테스트에서 best accuracy 달성하면 모델 저장
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        torch.save(model, os.path.join(workspace_path, f'cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt'))
        print(f'# save model: cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt\n')

print(f'\n\n# Best accuracy model({best_acc * 100:.2f}%): cifar10_cnn_model_best_acc_{best_epoch}-epoch.pt\n')

Train Epoch: 1 [0/50000 (0%)]	Average loss: 2.624154, Accuracy: 0.062500
Train Epoch: 1 [12800/50000 (26%)]	Average loss: 1.612992, Accuracy: 0.408815
Train Epoch: 1 [25600/50000 (51%)]	Average loss: 1.471965, Accuracy: 0.464386
Train Epoch: 1 [38400/50000 (77%)]	Average loss: 1.388374, Accuracy: 0.497296

Test set: Average loss: 1.1968, Accuracy: 0.563200

# save model: cifar10_cnn_model_best_acc_1-epoch.pt

Train Epoch: 2 [0/50000 (0%)]	Average loss: 0.970803, Accuracy: 0.625000
Train Epoch: 2 [12800/50000 (26%)]	Average loss: 1.053160, Accuracy: 0.625078
Train Epoch: 2 [25600/50000 (51%)]	Average loss: 1.037411, Accuracy: 0.630455
Train Epoch: 2 [38400/50000 (77%)]	Average loss: 1.015890, Accuracy: 0.637947

Test set: Average loss: 0.9406, Accuracy: 0.665300

# save model: cifar10_cnn_model_best_acc_2-epoch.pt

Train Epoch: 3 [0/50000 (0%)]	Average loss: 0.797038, Accuracy: 0.734375
Train Epoch: 3 [12800/50000 (26%)]	Average loss: 0.880217, Accuracy: 0.693952
Train Epoch: 3 [25600/5

Conv-BN-ReLU 레이어 3개 추가하기, epoch 20